<a href="https://colab.research.google.com/github/nunocesarsa/GoogleColab_public/blob/master/Agustin_data_exploring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("ola")

ola


# Passo 1: importar o GEE e o drive

In [2]:
#importing Earth Engine package
import ee

#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=C88FWegIfKCMpZRc8BFZQNnQwylBJTkA1Zo9l1Qkr0s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zQE8wwNcRQha6YO_3zJYU6jeyF4UfZ0ucv21n7yGe2KRQ18WH3681XE

Successfully saved authorization token.


In [3]:
#and now we link the colab also to the google drive so we can use it to store the outputs
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Carregar uma imagem do google baseado num poligno

In [14]:
#this is an area near portugal
poly_area_coord = [[[-8.899994101775464,40.17486696682209],
                    [-8.731079306853589,40.17486696682209],
                    [-8.731079306853589,40.305379406094474],
                    [-8.899994101775464,40.305379406094474],
                    [-8.899994101775464,40.17486696682209]]]



#This creates an earth engine polygon geometry
poly_area = ee.Geometry.Polygon(poly_area_coord)
tiles = ['29TNE']


#now we can use this, as a guide for selecting data
#check out the product table itself: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR
s2_collection = (ee.ImageCollection("COPERNICUS/S2_SR") #selects the L8 Surface reflectance product in GEE
                 .select(['B2','B3','B4',
                          'B5','B6','B7',
                          'B8A','B11','B12'])
                 .filter(ee.Filter.inList('MGRS_TILE', tiles)) #selects the bands of interest - in this case we are actually picking up all of them
                 .filter(ee.Filter.date('2020-01-01','2020-03-31')) #filters on the time - you can play around with this to select different dates..
                 .filterBounds(poly_area) #filters on the Aoi we have created above
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)))

#l8_collection = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") #selects the L8 Surface reflectance product in GEE
#                 .select(['B1','B2','B3','B4',
#                          'B5','B6','B7',
#                          'B10','B11']) #selects the bands of interest - in this case we are actually picking up all of them
#                 .filter(ee.Filter.date('2017-06-01','2017-06-30')) #filtes on the time - you can play around with this to select different dates..
#                 .filterBounds(poly_area) #filters on the Aoi we have created above
#                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
#                 )

count = s2_collection.size()
print('Nr of images in collection: ', str(count.getInfo())+'\n')

Nr of images in collection:  8



Passo seguinte: ver a imagem

In [0]:
mid_step = s2_collection.toList(count)
s2_img = ee.Image(mid_step.get(3))
s2_img_meta = s2_img.getInfo()
#s2_img_meta

s2_img_outra = ee.Image(mid_step.get(0))

#this command clips the image based on the original poly area
s2_img_clip = s2_img.clip(poly_area)

#first step now is to then generate a s2 image cropped to our aoi, and for information i will also plot it on the map

# Import the Folium library. This library is to assist in visualizations
import folium

# Import the Image function from the IPython.display module. 
from IPython.display import Image


#the next code snippet i stole from someone
# Define a method for displaying Earth Engine image tiles to folium map.

#this is a simplified version of the original function
def add_ee_layer(self, ee_image_object, vis_params, name):
  
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


# This first step sets up where is the "default starting position" of the map. And also the zoom
#my_map = folium.Map(location=[33,-80], zoom_start=8) #this had to be tried one by one... feels daft
my_map = folium.Map(location=[0,0], zoom_start=2) #this had to be tried one by one... feels daft

# Set visualization parameters. Here is we can create different visualizations of our data
visParams = {"bands": ['B4', 'B3', 'B2'],"gain": '0.1, 0.1, 0.1',"scale":20}
visParams_nir = {"bands": ['B8A', 'B4', 'B3'],"gain": '0.1, 0.1, 0.1',"scale":20}

# Add the image to the folium map
my_map.add_ee_layer(s2_img, visParams, 'RGB composite')
my_map.add_ee_layer(s2_img_outra, visParams, 'RGB composite - outra')
my_map.add_ee_layer(s2_img_clip, visParams_nir, 'NIR composite - clipped')
my_map.add_ee_layer(s2_img, visParams_nir, 'NIR composite')

ndvi = s2_img.normalizedDifference(['B8A', 'B4'])
ndviParams = {'min': -1, 'max': 1, 'palette': ['lightblue','cyan','blue','darkblue','brown','lightgreen','green','darkgreen']};
my_map.add_ee_layer(ndvi, ndviParams, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Recortar e gravaar para uma area de estudo

In [16]:
s2_img_meta.get('properties',{}).get('PRODUCT_ID')


'S2A_MSIL2A_20200219T112111_N0214_R037_T29TNE_20200219T124126'

In [0]:
#getting the filenames
imagename = s2_img_meta.get('properties',{}).get('PRODUCT_ID')


In [0]:
#these are task commands to be sent to GEE
task_config = {
    'image': s2_img,
    'fileFormat': 'GeoTIFF',
    'folder': 'Agustin',
    'fileNamePrefix': imagename,
    'description': "clipped area",
    'scale':20,
    'region':poly_area
}

#This is how we order it to start
task = ee.batch.Export.image.toDrive(**task_config )
task.start()

In [26]:
#to check if the task is running or completed, or failed, run this
task.status()

{'creation_timestamp_ms': 1588340917073,
 'description': 'clipped area',
 'destination_uris': ['https://drive.google.com/#folders/1iJxF_ZYLz9cGSXHVjhkyRXcZ6pR5YM7o'],
 'id': '3AJKK7RNWA5YSQE6P46Q7QYQ',
 'name': 'projects/earthengine-legacy/operations/3AJKK7RNWA5YSQE6P46Q7QYQ',
 'start_timestamp_ms': 1588340929871,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1588340942758}

# Descarregar em loop

Passo 1: criar a lista de imagens (collecçao)

In [28]:
#this is an area near portugal
poly_area_coord = [[[-8.899994101775464,40.17486696682209],
                    [-8.731079306853589,40.17486696682209],
                    [-8.731079306853589,40.305379406094474],
                    [-8.899994101775464,40.305379406094474],
                    [-8.899994101775464,40.17486696682209]]]



#This creates an earth engine polygon geometry
poly_area = ee.Geometry.Polygon(poly_area_coord)
tiles = ['29TNE']


#now we can use this, as a guide for selecting data
#check out the product table itself: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR
s2_collection = (ee.ImageCollection("COPERNICUS/S2_SR") #selects the L8 Surface reflectance product in GEE
                 .select(['B2','B3','B4',
                          'B5','B6','B7',
                          'B8A','B11','B12'])
                 .filter(ee.Filter.inList('MGRS_TILE', tiles)) #selects the bands of interest - in this case we are actually picking up all of them
                 .filter(ee.Filter.date('2020-01-01','2020-03-31')) #filters on the time - you can play around with this to select different dates..
                 .filterBounds(poly_area) #filters on the Aoi we have created above
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)))

count = s2_collection.size()
print('Nr of images in collection: ', str(count.getInfo())+'\n')

Nr of images in collection:  8



In [34]:
count.getInfo()

for i in range(count.getInfo()):
  print(i)
  s2_i = ee.Image(mid_step.get(i))
  s2_i_metadados = s2_i.getInfo()
  print(s2_i_metadados.get('properties',{}).get('PRODUCT_ID'))

  imagename = s2_i_metadados.get('properties',{}).get('PRODUCT_ID')

  #cria a tarefa
  task_config = {
    'image': s2_i,
    'fileFormat': 'GeoTIFF',
    'folder': 'Agustin',
    'fileNamePrefix': imagename,
    'description': "clipped area",
    'scale':20,
    'region':poly_area
  }
  #inicia a tarefa
  task = ee.batch.Export.image.toDrive(**task_config )
  task.start()

  #
  flag = task.status()['state']
  while flag != 'COMPLETED':
    flag = task.status()['state']
    #print(k,flag)
    if flag == 'FAILED':

      print("GEE task failed")
      flag = 'COMPLETED'

    



0
S2B_MSIL2A_20200105T112349_N0213_R037_T29TNE_20200105T123810
1
S2A_MSIL2A_20200110T112431_N0213_R037_T29TNE_20200110T123531
2
S2A_MSIL2A_20200120T112401_N0213_R037_T29TNE_20200120T123507
3
S2A_MSIL2A_20200219T112111_N0214_R037_T29TNE_20200219T124126
4
S2A_MSIL2A_20200310T112111_N0214_R037_T29TNE_20200310T123418
5
S2B_MSIL2A_20200318T113319_N0214_R080_T29TNE_20200318T142214
6
S2A_MSIL2A_20200323T113321_N0214_R080_T29TNE_20200323T160705
7
S2B_MSIL2A_20200328T113319_N0214_R080_T29TNE_20200328T142123
